In [1]:
from datetime import date
import requests
import pandas as pd
import time
pd.set_option('display.max_columns', None)


In [2]:
institution_ror = catalog.load('params:openalex_fetch_options.institution_ror')
#env = catalog.load('params:fetch_options.env')
env = 'dev'

                    INFO     Loading data from params:openalex_fetch_options.institution_ror    ]8;id=305120;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=238232;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [3]:
env

'dev'

In [ ]:
def openalex_fetch_author(institution_ror, env):
    cursor = '*'
    base_url = 'https://api.openalex.org/authors?filter=affiliations.institution.ror:{}&cursor={}'
    iteration_limit = 5
    iteration_count = 0

    url = base_url.format(institution_ror, cursor)
    api_response = requests.get(url).json()

    print(f'Iteration count: {iteration_count}')
    print(f'GET {url}')

    # creo dataframe con las columnas del primer resultado 
    df = pd.DataFrame.from_dict(api_response['results'])

    # update cursor
    cursor = api_response['meta']['next_cursor']
    url = base_url.format(institution_ror, cursor)

    while cursor:

        if env == 'dev' and iteration_count >= iteration_limit:
            break

        # request api with updated cursor
        url = base_url.format(institution_ror, cursor)

        iteration_count += 1
        print(f'Iteration count: {iteration_count}')
        print(f'GET {url}')

        api_response = requests.get(url).json()

        df_tmp = pd.DataFrame.from_dict(api_response['results'])

        df = pd.concat([df, df_tmp])

        # update cursor
        cursor = api_response['meta']['next_cursor']
        df['load_datetime'] = date.today()

    return df

In [ ]:
df = openalex_fetch_author(institution_ror, env)

Iteration count: 0
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/01tjs6929&cursor=*
Iteration count: 1
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/01tjs6929&cursor=Ils5NDYsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTEwMjAxNTE0NCddIg==
Iteration count: 2
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/01tjs6929&cursor=Ils2MzYsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTEwNTUyOTA2OCddIg==
Iteration count: 3
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/01tjs6929&cursor=Ils1NDUsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTEwMTc4MzY4NSddIg==
Iteration count: 4
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/01tjs6929&cursor=Ils0ODMsICdodHRwczovL29wZW5hbGV4Lm9yZy9BNTA1MDYzMjg1OCddIg==
Iteration count: 5
GET https://api.openalex.org/authors?filter=affiliations.institution.ror:https://ror.org/01tjs6929&cursor=Ils0

In [6]:
df

,id,orcid,display_name,display_name_alternatives,works_count,cited_by_count,summary_stats,ids,affiliations,last_known_institutions,topics,topic_share,x_concepts,counts_by_year,works_api_url,updated_date,created_date,load_datetime
0,https://openalex.org/A5013845636,https://orcid.org/0000-0002-6764-4789,P. H. Hansen,"[J B. Hansen, M. C. Hansen, J R H Hansen, Fran...",2887,133693,"{'2yr_mean_citedness': 2.525911708253359, 'h_i...",{'openalex': 'https://openalex.org/A5013845636...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I124055696', 'ro...","[{'id': 'https://openalex.org/T10048', 'displa...","[{'id': 'https://openalex.org/T10527', 'displa...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2025, 'works_count': 68, 'oa_works_c...",https://api.openalex.org/works?filter=author.i...,2025-09-07T11:01:14.374671,2023-07-21,2025-09-08
1,https://openalex.org/A5049875695,https://orcid.org/0000-0002-1653-1303,M. Erdmann,"[M. Erdmann, W. Erdmann, J. Erdmann, Martin Er...",2675,136872,"{'2yr_mean_citedness': 3.5545722713864305, 'h_...",{'openalex': 'https://openalex.org/A5049875695...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I887968799', 'ro...","[{'id': 'https://openalex.org/T10048', 'displa...","[{'id': 'https://openalex.org/T10527', 'displa...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2025, 'works_count': 67, 'oa_works_c...",https://api.openalex.org/works?filter=author.i...,2025-09-02T11:40:42.494453,2023-07-21,2025-09-08
2,https://openalex.org/A5046725106,https://orcid.org/0000-0001-7951-0166,C. Padilla Aranda,"[Carolina Rodríguez Padilla, C. Padilla, C. Pa...",2280,125614,"{'2yr_mean_citedness': 4.683139534883721, 'h_i...",{'openalex': 'https://openalex.org/A5046725106...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210132884', 'r...","[{'id': 'https://openalex.org/T10048', 'displa...","[{'id': 'https://openalex.org/T10527', 'displa...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2025, 'works_count': 89, 'oa_works_c...",https://api.openalex.org/works?filter=author.i...,2025-09-01T14:05:29.806983,2023-07-21,2025-09-08
3,https://openalex.org/A5087325828,https://orcid.org/0000-0002-3021-0258,F. Barreiro,"[F. Alonso, F H Barreiro, J. Costa, J. Barreir...",2178,122889,"{'2yr_mean_citedness': 4.641025641025641, 'h_i...",{'openalex': 'https://openalex.org/A5087325828...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I63634437', 'ror...","[{'id': 'https://openalex.org/T10048', 'displa...","[{'id': 'https://openalex.org/T10527', 'displa...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2025, 'works_count': 62, 'oa_works_c...",https://api.openalex.org/works?filter=author.i...,2025-08-29T22:33:14.140360,2023-07-21,2025-09-08
4,https://openalex.org/A5062435541,https://orcid.org/0000-0001-6322-6195,A. T. Doyle,"[Tony Doyle, Anthony T. Doyle, A. Doyle, T. Do...",2175,117700,"{'2yr_mean_citedness': 3.6802325581395348, 'h_...",{'openalex': 'https://openalex.org/A5062435541...,[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I7882870', 'ror'...","[{'id': 'https://openalex.org/T10048', 'displa...","[{'id': 'https://openalex.org/T10527', 'displa...","[{'id': 'https://openalex.org/C121332964', 'wi...","[{'year': 2025, 'works_count': 65, 'oa_works_c...",https://api.openalex.org/works?filter=author.i...,2025-09-08T09:37:28.667590,2023-07-21,2025-09-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,https://openalex.org/A5027210888,None,Virpi Sinikka Niemelä,"[V. S. Niemelä, Virpi Sinikka Niemelä, V. S. N...",395,2044,"{'2yr_mean_citedness': 0.0, 'h_index': 26, 'i1...",{'openalex': 'https://openalex.org/A5027210888'},[{'institution': {'id': 'https://openalex.org/...,"[{'id': 'https://openalex.org/I4210108180', 'r...","[{'id': 'https://openalex.org/T14163', 'displa...","[{'id': 